<a href="https://colab.research.google.com/github/TasnimJumara/TableAssignmentFirst/blob/master/Assignment3_170104046.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import torch
import torch.nn as nn
import torchtext
import time
import random
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import DataLoader, Dataset

In [4]:
dataset=pd.read_csv(r'/content/sample_data/Dataset 2.csv', encoding = "ISO-8859-1")
dataset.head()

,text,polarity
0,just had a real good moment. i missssssssss hi...,0
1,is reading manga http://plurk.com/p/mzp1e,0
2,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,@lapcat Need to send 'em to my accountant tomo...,0
4,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [5]:
dataset.value_counts()

text                                                                                                                                                                          polarity
Mom's depression tied to kids' emotional, intellectual development  https://ift.tt/2HtVZCEÃÂ                                                                                 1           11
More than 1 in 20 US children and teens have anxiety or depression  https://www.sciencedaily.com/releases/2018/04/180424184119.htmÃÂ Ã¢ÂÂ¦                                  1            4
25-04-18                                                                                                                                                                      1            4
[Zicutake] Depression can be prevented by exercise  https://goo.gl/fb/Mu7k6tÃÂ                                                                                               1            3
Good morning                                                 

In [ ]:
dataset.describe()


,polarity
count,10314.000000
mean,0.224355
std,0.417177
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
dataset.shape

(10314, 2)

In [ ]:
dataset[1:15]

,text,polarity
1,is reading manga http://plurk.com/p/mzp1e,0
2,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,@lapcat Need to send 'em to my accountant tomo...,0
4,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
5,so sleepy. good times tonight though,0
6,"@SilkCharm re: #nbn as someone already said, d...",0
7,23 or 24Ã¯Â¿Â½C possible today. Nice,0
8,nite twitterville workout in the am -ciao,0
9,"@daNanner Night, darlin'! Sweet dreams to you",0
10,Good morning everybody!,0


In [20]:
class Sequences(Dataset):
    def __init__(self, data):
        self.vectorizer = CountVectorizer(stop_words='english')
        self.sequences = self.vectorizer.fit_transform(data.text.tolist())
        self.labels = data.polarity.tolist()
        self.token2idx = self.vectorizer.vocabulary_
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
    
    def __getitem__(self, i):
        return self.sequences[i, :].toarray(), self.labels[i]
    
    def __len__(self):
        return self.sequences.shape[0]

In [13]:
print(len(dataset))

10314


In [14]:
train_set, test_set = train_test_split(dataset, train_size=0.8, test_size=0.2, random_state=1)

In [21]:
data = Sequences(train_set)

In [15]:
# Hyper-parameters
sequence_length = 128
input_size = 128
hidden_size = 128
num_layers = 3
num_classes = 2
batch_size = 100
num_epochs = 1
learning_rate = 0.01

In [16]:
train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_set,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [24]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.rnn(x, (h0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [26]:
model = RNN(input_size, hidden_size, num_layers, num_classes)


In [27]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (text, polarity) in enumerate(train_loader):
        texts = text.to(device)
        label = polarity.to(device)
        
        # Forward pass
        outputs = model(texts)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for text, polaity in test_loader:
        texts = text.to(device)
        labels = polarity.to(device)
        outputs = model(texts)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model Is: {} %'.format(100 * correct / total)) 


Epoch [1/1], Step [100/600], Loss: 2.1194
Epoch [1/1], Step [200/600], Loss: 1.6383
Epoch [1/1], Step [300/600], Loss: 1.3558
Epoch [1/1], Step [400/600], Loss: 1.4002
Epoch [1/1], Step [500/600], Loss: 1.1695
Epoch [1/1], Step [600/600], Loss: 1.2021
Test Accuracy of the model Is: 64.13 %


In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def predict_polarity(text):
  test_vector = torch.LongTensor(dataset.vectorizer.transform([text]).toarray())
  
  output = model(test_vector)
  
  prediction = torch.sigmoid(output).item()
  
  if prediction > 0.7:
    print(f'{prediction:0.3}: Positive Polarity')
    return 1
  else:
    print(f'{prediction:0.3}: Negative Polarity')
    return 0

In [ ]:
pred_labels = []

sentences = list(df_test['text'])
labels = df_test['polarity']

print(sentences)

for sentence in sentences:
  pred_labels.append(predict_polarity(sentence))

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(labels, pred_labels)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(labels, pred_labels)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(labels, pred_labels)
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(labels, pred_labels)
print('F1 score: %f' % f1)

# confusion matrix
matrix = confusion_matrix(labels, pred_labels)
print(matrix)